In [1]:
import os
import codecs
import yaml

import rqalpha as rqa
import rqdatac as rqd
import rqoptimizer as rqo

import pandas as pd
import numpy as np

import utils
import multiprocessing
import time
%matplotlib inline

In [2]:
# def _init_rqdata(conf_path):
#     with codecs.open(conf_path, 'r', encoding='utf8') as stream:
#         conf = yaml.load(stream)
#         if 'proxy' in conf:
#             proxy_info = (conf['proxy'].get('type'),
#                 conf['proxy'].get('host'),
#                 conf['proxy'].get('port'),
#                 conf['proxy'].get('user'),
#                 conf['proxy'].get('password'))
#         else:
#             proxy_info = None
#         rqd.init(conf['rqdata_username'], conf['rqdata_password'],
#             (conf['rqdata_host'], conf['rqdata_port']), proxy_info=proxy_info)

# _init_rqdata('rqpro.yml')

In [3]:
rqd.init()

F:\anaconda\lib\site-packages\rqdatac\client.py:190: UserWarning: rqdatac is already inited. Settings will be changed.
  warnings.warn("rqdatac is already inited. Settings will be changed.", stacklevel=0)


In [4]:
# backtest related
def get_target_stocks(context, factor, ascending, percent_selected, min_selected, industry):
    """获得当天需要买入的股票"""
    # 前一个交易日
    selection_date = rqd.get_previous_trading_date(context.now)
    if context.industry is not None:
        pass
    else:
        # 前一个交易日的指数成分股
        universe = rqd.index_components(context.index_stockpool, selection_date)
    # 剔除当天停牌
    universe = utils.drop_suspended(universe, selection_date)
    # 剔除当天ST
    universe = utils.drop_st(universe, selection_date)
    # 剔除上市小于60天
    universe = utils.drop_recently_listed(universe, selection_date, 60)
    # 获得因子分数
    scores = utils.get_factor(universe, factor, selection_date, selection_date)
    # 返回需要买入的股票
    return utils.select_top_N_percent(
        universe, selection_date, scores, percent_selected, ascending, min_selected,
        grouper=utils.get_industry,
    )


def get_target_portfolio(context, **optimization_args):
    # 返回每只股票的权重Series
    # rqoptimizer 默认取date前一个交易日的数据
    target_portfolio = rqo.portfolio_optimize(
        context.target_stocks, context.now, **optimization_args
    )
    return target_portfolio.loc[lambda x: x != 0]



def _should_rebalance(context):
    """判断今天是否调仓"""
    # 获得上一个交易日
    prev_trading_day = rqd.get_previous_trading_date(context.now)
    # 判断是否是月初, 是月初返回True
    is_month_start = (prev_trading_day.month != context.now.month)
    return is_month_start

def rebalance(context, bar_dict):
    # 先清空不在目标组合里面的股票
    positions = context.stock_account.positions
    for order_book_id in positions:
        if order_book_id not in context.target_portfolio:
            rqa.api.order_to(order_book_id, 0)

    # 对每个股票计算目标权重和当前权重的差值
    # 差值为正的是买单, 反之为卖单
    capital = context.stock_account.total_value * (1 - context.cash_cushion)
    to_sell, to_buy = {}, {}
    for order_book_id, weight in context.target_portfolio.items():
        # 股票目标价值
        target_value = capital * weight
        # 目标和现有之差
        gap = target_value - positions[order_book_id].market_value
        if gap > 0:
            to_buy[order_book_id] = gap
        else:
            to_sell[order_book_id] = gap

    # to avoid liquidity issue, sell first, buy second
    _money_for_one_lot = lambda order_book_id: bar_dict[order_book_id].close * 100
    # 先卖
    for order_book_id, value in to_sell.items():
        # 要卖出的至少大于1手股票价值
        if abs(value) > _money_for_one_lot(order_book_id):
            # value为负
            rqa.api.order_value(order_book_id, value)
    # 后买
    for order_book_id, value in to_buy.items():
        # 要买入的至少大于1手股票价值
        if abs(value) > _money_for_one_lot(order_book_id):
            rqa.api.order_value(order_book_id, value)
            
def init(context):
    pass     


def handle_bar(context, bar_dict):
    # 如果是False(不是月初),直接返回
    if not _should_rebalance(context):
        return
    print(context.now)
    context.target_stocks = get_target_stocks(context, **context.stock_selection_args)
    context.target_portfolio = get_target_portfolio(context, **context.optimization_args)
    rebalance(context, bar_dict)


In [5]:
MILLION = 1_000_000
BILLION = 1000 * MILLION

config = {
    "base": {
        "matching_type": "current_bar",
        "start_date": '2020-01-01',
        "end_date": '2020-07-20',
        "frequency": '1d',
        "accounts": {"stock": 0.1 * BILLION},
    },

    "mod": {
        "sys_analyser": {
            "enabled": True, 
            "plot": True,  
            "benchmark": "000905.XSHG",
        },
    },

    "extra": {
        "log_level": 'info',
        "context_vars": {
            'cash_cushion': 0.005,
            'benchmark_symbol': '000985.XSHG',
            'industry': None,
            'stock_selection_args': {
                'factor': None,
                'ascending': True ,
                'percent_selected': 0.1,
                'min_selected': 10,
            },

            'optimization_args': {
                'benchmark': '000985.XSHG',
                'objective': rqo.MinTrackingError(),
                'cons': [rqo.WildcardIndustryConstraint(lower_limit=-0.05, upper_limit=0.05,relative=True)],
                'bnds': {'*': (0, 0.1)},
            },
        },
    },
}

In [6]:
backtest_result = rqa.run_func(init=init,handle_bar=handle_bar, config=config)

[2020-01-02 15:00:00.000000] INFO: user_log: 2020-01-02 15:00:00
[2020-01-02 15:00:00.000000] ERROR: user_system_log: 策略运行产生异常
Traceback (most recent call last):
  File "F:\anaconda\lib\site-packages\rqalpha\core\strategy.py", line 97, in handle_bar
    self._handle_bar(self._user_context, bar_dict)
  File "<ipython-input-4-84d5e3cad6d3>", line 88, in handle_bar
    context.target_stocks = get_target_stocks(context, **context.stock_selection_args)
TypeError: get_target_stocks() missing 1 required positional argument: 'industry'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "F:\anaconda\lib\site-packages\rqalpha\main.py", line 207, in run
    executor.run(bar_dict)
  File "F:\anaconda\lib\site-packages\rqalpha\core\executor.py", line 47, in run
    self._split_and_publish(event)
  File "F:\anaconda\lib\site-packages\rqalpha\core\executor.py", line 98, in _split_and_publish
    self._env.event_bus.publish_event(e)
  File "

----------

In [7]:
# performance analysis

def cumulative_excess_return(backtest_result):
    """累计超额收益"""
    # 投资组合中每日现金、权益、市值、净值等数据
    strat_portf = backtest_result['sys_analyser']['portfolio']
    benchmark_portf = backtest_result['sys_analyser']['benchmark_portfolio']
    # 单位净值计算出的每日收益率
    strat_return = strat_portf['unit_net_value'].pct_change()
    # benchmark的每日收益率
    benchmark_return = benchmark_portf['unit_net_value'].pct_change()
    # 每日超额收益率
    excess_return = strat_return - benchmark_return
    
    return (excess_return).cumsum()+1
    

def average_turnover(backtest_result):
    sell_trades = backtest_result['sys_analyser']['trades'].query("side == 'SELL'")
    portfolio = backtest_result['sys_analyser']['portfolio']
    
    total_trade_value = lambda df: (df['last_price'] * df['last_quantity']).sum()
    value_sold = sell_trades.groupby(sell_trades.index, group_keys=False).apply(total_trade_value)
    value_sold.index = pd.to_datetime(value_sold.index).date
    
    capital = portfolio.loc[value_sold.index, 'total_value']
    
    return (value_sold / capital).mean()

def stock_position_stats(backtest_result):
    market_value = backtest_result['sys_analyser']['stock_account']['market_value']
    total_value = backtest_result['sys_analyser']['stock_account']['total_value']
    percentage = market_value / total_value
    num_holdings = backtest_result['sys_analyser']['stock_positions'].groupby(level='date').size()
    return pd.DataFrame(
        {'market_value': market_value, 
         'total_value': total_value,
         'percentage': percentage,
         'num_holdings': num_holdings,
        })

In [8]:
cumulative_excess_return(backtest_result).plot(title='Cumulative Excess Return', figsize=(18, 6))
print(f'Average Turnover: {average_turnover(backtest_result)}')

TypeError: 'NoneType' object is not subscriptable

In [ ]:
stock_position_stats(backtest_result)['percentage'].plot(figsize=(18, 6))

In [ ]:
stock_position_stats(backtest_result)['num_holdings'].plot(figsize=(18, 6))

In [ ]:
rqd.get_industry('医药', source='citics', date=None, market='cn')